In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random
from torchsummary import summary 
import onnx 
import onnxruntime as ort
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_absolute_error
import random
import os
import datetime
import matplotlib.pyplot as plt

In [3]:
import os, sys
REPO = "/home/deepaksr/project/Project_files_2/U-2-Net"    
os.chdir(REPO)
sys.path.insert(0, REPO)

In [4]:
from model import U2NET, U2NETP

In [5]:
class Saliency_RGB_v1(nn.Module):
    def __init__(self):
        super(Saliency_RGB_v1, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(64, 1, kernel_size=2, stride=2), nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [6]:
class Saliency_RGB_v2(nn.Module):
    def __init__(self):
        super(Saliency_RGB_v2, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(8, 16, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(16, 8, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(8, 1, kernel_size=2, stride=2), nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
# ======== CONFIG ========
# Input image folder
input_folder = '/home/deepaksr/project/Saliency_datasets/co-reg/simulated/images/RGB'

# Output base folder
output_base_folder = '/home/deepaksr/project/Saliency_datasets/co-reg/simulated/rgb_raw_predictions'

# List of models with (name, path)
models_info = [
    ('u2net', '/home/deepaksr/project/Project_files_2/U-2-Net/training_logs/u2netp_finetune_rgb_d1_20250511_201128/best_model.pth'),
    ('sal_v1', '/home/deepaksr/project/Project_files_2/training_logs/RGB_v1_d1_fixed_bce_20250514_081726/best_model.pth'),
    ('sal_v2', '/home/deepaksr/project/Project_files_2/training_logs/RGB_v2_d1_fixed_bce_20250514_090812/best_model.pth'),
]

# Image size expected by the model
image_size = (320, 320)
# =========================

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Image transforms
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
])

# Prediction saving function
def save_prediction(output_tensor, save_path):
    output = output_tensor.squeeze().cpu().detach().numpy()
    output = (output * 255).astype(np.uint8)
    Image.fromarray(output).save(save_path)

# Load and run inference for each model
for model_name, model_path in models_info:
    print(f"Processing with model: {model_name}")

    # Load model and weights
    model = U2NET(3, 1)  # Replace with your specific model class if different
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    # Create output subfolder for this model
    model_output_folder = os.path.join(output_base_folder, model_name)
    os.makedirs(model_output_folder, exist_ok=True)

    # Process all images
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            image_path = os.path.join(input_folder, filename)
            image = Image.open(image_path).convert('RGB')
            input_tensor = transform(image).unsqueeze(0).to(device)

            with torch.no_grad():
                d_output = model(input_tensor)
                # If model returns multiple outputs (U²-Net does), take the first
                if isinstance(d_output, (list, tuple)):
                    d_output = d_output[0]

            save_path = os.path.join(model_output_folder, filename)
            save_prediction(d_output, save_path)

    print(f"Saved predictions for {model_name} in {model_output_folder}")

print("✅ All model predictions saved.")


In [7]:
import os
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import numpy as np
from model import U2NET, U2NETP

# === Define Custom IR Models ===
class Saliency_RGB_v1(nn.Module):
    def __init__(self):
        super(Saliency_RGB_v1, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(64, 1, kernel_size=2, stride=2), nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
class Saliency_RGB_v2(nn.Module):
    def __init__(self):
        super(Saliency_RGB_v2, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(8, 16, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(16, 8, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(8, 1, kernel_size=2, stride=2), nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# === CONFIG ===
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_folder = '/home/deepaksr/project/Saliency_datasets/co-reg/simulated/images/RGB'
output_base_folder = '/home/deepaksr/project/Saliency_datasets/co-reg/simulated/rgb_raw_predictions'
image_size = (320, 320)

# === Define model loading info ===
models_info = {
    'u2net': {
        'model_class': lambda: U2NETP(3, 1),
        'weight_path': '/home/deepaksr/project/Project_files_2/U-2-Net/training_logs/u2netp_finetune_rgb_d1_20250511_201128/best_model.pth',
        'input_mode': 'RGB'
    },
    'sal_v1': {
        'model_class': Saliency_RGB_v1,
        'weight_path': '/home/deepaksr/project/Project_files_2/training_logs/RGB_v1_d1_fixed_bce_20250514_081726/best_model.pth',
        'input_mode': 'RGB'
    },
    'sal_v2': {
        'model_class': Saliency_RGB_v2,
        'weight_path': '/home/deepaksr/project/Project_files_2/training_logs/RGB_v2_d1_fixed_bce_20250514_090812/best_model.pth',
        'input_mode': 'RGB'
    }
}

# === Transform Function ===
def get_transform(input_mode):
    return transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor()
    ])

# === Save prediction ===
def save_prediction(output_tensor, save_path):
    output = output_tensor.squeeze().cpu().detach().numpy()
    output = (output * 255).astype(np.uint8)
    Image.fromarray(output).save(save_path)

# === Inference Loop ===
for model_name, info in models_info.items():
    print(f"\n🔄 Running inference for: {model_name}")
    
    model = info['model_class']().to(device)
    model.load_state_dict(torch.load(info['weight_path'], map_location=device))
    model.eval()
    
    # Set correct image mode (L for grayscale)
    input_mode = info['input_mode']
    transform = get_transform(input_mode)

    # Prepare output directory
    model_output_folder = os.path.join(output_base_folder, model_name)
    os.makedirs(model_output_folder, exist_ok=True)

    for fname in os.listdir(input_folder):
        if fname.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp')):
            img_path = os.path.join(input_folder, fname)
            image = Image.open(img_path).convert(input_mode)
            input_tensor = transform(image).unsqueeze(0).to(device)

            with torch.no_grad():
                output = model(input_tensor)
                if isinstance(output, (list, tuple)):
                    output = output[0]

            save_path = os.path.join(model_output_folder, fname)
            save_prediction(output, save_path)

    print(f"✅ Saved predictions in {model_output_folder}")

print("\n🎉 All model predictions completed.")



🔄 Running inference for: u2net


/tmp/ipykernel_2525057/1559468262.py:93: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(info['weight_path'], map_location=device))
/home/deep

✅ Saved predictions in /home/deepaksr/project/Saliency_datasets/co-reg/simulated/rgb_raw_predictions/u2net

🔄 Running inference for: sal_v1
✅ Saved predictions in /home/deepaksr/project/Saliency_datasets/co-reg/simulated/rgb_raw_predictions/sal_v1

🔄 Running inference for: sal_v2
✅ Saved predictions in /home/deepaksr/project/Saliency_datasets/co-reg/simulated/rgb_raw_predictions/sal_v2

🎉 All model predictions completed.


In [10]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF

# Folder paths
gt_folder = '/home/deepaksr/project/Saliency_datasets/co-reg/simulated/saliency_masks'  # Update with actual GT path
binary_base_folder = output_base_folder.replace("rgb_raw_predictions", "rgb_binary_mask_pred")
comparison_base_folder = output_base_folder.replace("rgb_raw_predictions", "rgb_comparison")

titles = ['Input Image', 'Ground Truth', 'Raw Prediction', 'Binary Prediction']

for model_name in models_info:
    print(f"Post-processing outputs for model: {model_name}")

    raw_folder = os.path.join(output_base_folder, model_name)
    binary_folder = os.path.join(binary_base_folder, model_name)
    comparison_folder = os.path.join(comparison_base_folder, model_name)

    os.makedirs(binary_folder, exist_ok=True)
    os.makedirs(comparison_folder, exist_ok=True)

    for filename in os.listdir(raw_folder):
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue

        raw_path = os.path.join(raw_folder, filename)
        raw_pred = Image.open(raw_path).convert('L')
        raw_np = np.array(raw_pred) / 255.0
        binary_np = (raw_np > 0.5).astype(np.uint8) * 255
        binary_img = Image.fromarray(binary_np)

        binary_path = os.path.join(binary_folder, filename)
        binary_img.save(binary_path)

        img_filename = os.path.splitext(filename)[0] + '.jpg'
        image_path = os.path.join(input_folder, img_filename)
        gt_filename = os.path.splitext(filename)[0] + '.jpg'
        gt_path = os.path.join(gt_folder, gt_filename)

        if not os.path.exists(gt_path):
            print(f"⚠️ Ground truth mask not found for {gt_filename}, skipping visualization.")
            continue

        # Load and resize all images (grayscale)
        image = Image.open(image_path).convert('RGB').resize(image_size)
        gt = Image.open(gt_path).convert('L').resize(image_size)
        raw = raw_pred.resize(image_size)
        binary = binary_img.resize(image_size)

        images = [np.array(image), np.array(gt), np.array(raw), np.array(binary)]

        fig, axes = plt.subplots(1, 4, figsize=(12, 3))
        for ax, img, title in zip(axes, images, titles):
            # Use color map only for grayscale images
            if img.ndim == 2 or img.shape[2] == 1:
                ax.imshow(img, cmap='gray', vmin=0, vmax=255)
            else:
                ax.imshow(img)
            ax.set_title(title, fontsize=10)
            ax.axis('off')



        plt.tight_layout()
        save_path = os.path.join(comparison_folder, filename)
        plt.savefig(save_path, bbox_inches='tight', dpi=150)
        plt.close()

    print(f"✅ Saved binary predictions to {binary_folder}")
    print(f"✅ Saved visual comparisons to {comparison_folder}")


Post-processing outputs for model: u2net
✅ Saved binary predictions to /home/deepaksr/project/Saliency_datasets/co-reg/simulated/rgb_binary_mask_pred/u2net
✅ Saved visual comparisons to /home/deepaksr/project/Saliency_datasets/co-reg/simulated/rgb_comparison/u2net
Post-processing outputs for model: sal_v1
✅ Saved binary predictions to /home/deepaksr/project/Saliency_datasets/co-reg/simulated/rgb_binary_mask_pred/sal_v1
✅ Saved visual comparisons to /home/deepaksr/project/Saliency_datasets/co-reg/simulated/rgb_comparison/sal_v1
Post-processing outputs for model: sal_v2
✅ Saved binary predictions to /home/deepaksr/project/Saliency_datasets/co-reg/simulated/rgb_binary_mask_pred/sal_v2
✅ Saved visual comparisons to /home/deepaksr/project/Saliency_datasets/co-reg/simulated/rgb_comparison/sal_v2


In [11]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Folder paths
input_folder = '/home/deepaksr/project/Saliency_datasets/co-reg/simulated/images/RGB'
gt_folder = '/home/deepaksr/project/Saliency_datasets/co-reg/simulated/saliency_masks'
u2netp_folder = '/home/deepaksr/project/Saliency_datasets/co-reg/simulated/rgb_raw_predictions/u2net'
sal_v1_folder = '/home/deepaksr/project/Saliency_datasets/co-reg/simulated/rgb_raw_predictions/sal_v1'
sal_v2_folder = '/home/deepaksr/project/Saliency_datasets/co-reg/simulated/rgb_raw_predictions/sal_v2'

# Output folder
comparison_grid_folder = '/home/deepaksr/project/Saliency_datasets/co-reg/simulated/rgb_comparison_across_models'
os.makedirs(comparison_grid_folder, exist_ok=True)

# Titles for subplots
titles = ['Input Image', 'Ground Truth', 'U²-NetP', 'Saliency_v1', 'Saliency_v2']

# Image resize
image_size = (320, 320)

for filename in os.listdir(u2netp_folder):
    if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        continue

    base_name = os.path.splitext(filename)[0]
    input_path = os.path.join(input_folder, base_name + '.jpg')  # adjust extension if needed
    gt_path = os.path.join(gt_folder, base_name + '.jpg')        # adjust extension if needed
    u2netp_path = os.path.join(u2netp_folder, filename)
    sal_v1_path = os.path.join(sal_v1_folder, filename)
    sal_v2_path = os.path.join(sal_v2_folder, filename)

    if not (os.path.exists(input_path) and os.path.exists(gt_path) and 
            os.path.exists(u2netp_path) and os.path.exists(sal_v1_path) and os.path.exists(sal_v2_path)):
        print(f"⚠️ Missing files for {filename}, skipping.")
        continue

    # Load input image as RGB, others as grayscale
    input_img = np.array(Image.open(input_path).convert('RGB').resize(image_size))
    gt_img = np.array(Image.open(gt_path).convert('L').resize(image_size))
    u2netp_img = np.array(Image.open(u2netp_path).convert('L').resize(image_size))
    sal_v1_img = np.array(Image.open(sal_v1_path).convert('L').resize(image_size))
    sal_v2_img = np.array(Image.open(sal_v2_path).convert('L').resize(image_size))

    images = [input_img, gt_img, u2netp_img, sal_v1_img, sal_v2_img]

    # Create subplot
    fig, axes = plt.subplots(1, 5, figsize=(15, 3))
    for ax, img, title in zip(axes, images, titles):
        if img.ndim == 2:  # Grayscale
            ax.imshow(img, cmap='gray', vmin=0, vmax=255)
        else:  # RGB
            ax.imshow(img)
        ax.set_title(title, fontsize=9)
        ax.axis('off')

    plt.tight_layout()
    save_path = os.path.join(comparison_grid_folder, filename)
    plt.savefig(save_path, bbox_inches='tight', dpi=150)
    plt.close()

print(f"✅ Saved model comparison grids to {comparison_grid_folder}")

✅ Saved model comparison grids to /home/deepaksr/project/Saliency_datasets/co-reg/simulated/rgb_comparison_across_models


data 2

In [5]:
import os
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import numpy as np
from model import U2NET, U2NETP

# === Define Custom IR Models ===
class Saliency_RGB_v1(nn.Module):
    def __init__(self):
        super(Saliency_RGB_v1, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(64, 1, kernel_size=2, stride=2), nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
class Saliency_RGB_v2(nn.Module):
    def __init__(self):
        super(Saliency_RGB_v2, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(8, 16, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(16, 8, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(8, 1, kernel_size=2, stride=2), nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# === CONFIG ===
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_folder = '/home/deepaksr/project/Saliency_datasets/RGB_only/images'
output_base_folder = '/home/deepaksr/project/Saliency_datasets/RGB_only/rgb_raw_predictions'
image_size = (320, 320)

# === Define model loading info ===
models_info = {
    'u2net': {
        'model_class': lambda: U2NETP(3, 1),
        'weight_path': '/home/deepaksr/project/Project_files_2/U-2-Net/training_logs/u2netp_finetune_rgb_d3_20250511_212906/best_model.pth',
        'input_mode': 'RGB'
    },
    'sal_v1': {
        'model_class': Saliency_RGB_v1,
        'weight_path': '/home/deepaksr/project/Project_files_2/training_logs/RGB_v1_d3_fixed_bce_20250517_153240/best_model.pth',
        'input_mode': 'RGB'
    },
    'sal_v2': {
        'model_class': Saliency_RGB_v2,
        'weight_path': '/home/deepaksr/project/Project_files_2/training_logs/RGB_v2_d3_fixed_bce_20250518_022557/best_model.pth',
        'input_mode': 'RGB'
    }
}

# === Transform Function ===
def get_transform(input_mode):
    return transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor()
    ])

# === Save prediction ===
def save_prediction(output_tensor, save_path):
    output = output_tensor.squeeze().cpu().detach().numpy()
    output = (output * 255).astype(np.uint8)
    Image.fromarray(output).save(save_path)

# === Inference Loop ===
for model_name, info in models_info.items():
    print(f"\n🔄 Running inference for: {model_name}")
    
    model = info['model_class']().to(device)
    model.load_state_dict(torch.load(info['weight_path'], map_location=device))
    model.eval()
    
    # Set correct image mode (L for grayscale)
    input_mode = info['input_mode']
    transform = get_transform(input_mode)

    # Prepare output directory
    model_output_folder = os.path.join(output_base_folder, model_name)
    os.makedirs(model_output_folder, exist_ok=True)


    # Sort the filenames to ensure consistent ordering
    all_filenames = sorted([
        fname for fname in os.listdir(input_folder)
        if fname.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp'))
    ])

    # Select one image every 150 steps
    selected_filenames = all_filenames[::15]

    for fname in selected_filenames:




    #for fname in os.listdir(input_folder):
        if fname.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp')):
            img_path = os.path.join(input_folder, fname)
            image = Image.open(img_path).convert(input_mode)
            input_tensor = transform(image).unsqueeze(0).to(device)

            with torch.no_grad():
                output = model(input_tensor)
                if isinstance(output, (list, tuple)):
                    output = output[0]

            save_path = os.path.join(model_output_folder, fname)
            save_prediction(output, save_path)

    print(f"✅ Saved predictions in {model_output_folder}")

print("\n🎉 All model predictions completed.")



🔄 Running inference for: u2net


/tmp/ipykernel_2576842/1392970907.py:93: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(info['weight_path'], map_location=device))
/home/deep

✅ Saved predictions in /home/deepaksr/project/Saliency_datasets/RGB_only/rgb_raw_predictions/u2net

🔄 Running inference for: sal_v1
✅ Saved predictions in /home/deepaksr/project/Saliency_datasets/RGB_only/rgb_raw_predictions/sal_v1

🔄 Running inference for: sal_v2
✅ Saved predictions in /home/deepaksr/project/Saliency_datasets/RGB_only/rgb_raw_predictions/sal_v2

🎉 All model predictions completed.


In [6]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF

# Folder paths
gt_folder = '/home/deepaksr/project/Saliency_datasets/RGB_only/saliency_masks'  # Update with actual GT path
binary_base_folder = output_base_folder.replace("rgb_raw_predictions", "rgb_binary_mask_pred")
comparison_base_folder = output_base_folder.replace("rgb_raw_predictions", "rgb_comparison")

titles = ['Input Image', 'Ground Truth', 'Raw Prediction', 'Binary Prediction']

for model_name in models_info:
    print(f"Post-processing outputs for model: {model_name}")

    raw_folder = os.path.join(output_base_folder, model_name)
    binary_folder = os.path.join(binary_base_folder, model_name)
    comparison_folder = os.path.join(comparison_base_folder, model_name)

    os.makedirs(binary_folder, exist_ok=True)
    os.makedirs(comparison_folder, exist_ok=True)

    for filename in os.listdir(raw_folder):
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue

        raw_path = os.path.join(raw_folder, filename)
        raw_pred = Image.open(raw_path).convert('L')
        raw_np = np.array(raw_pred) / 255.0
        binary_np = (raw_np > 0.5).astype(np.uint8) * 255
        binary_img = Image.fromarray(binary_np)

        binary_path = os.path.join(binary_folder, filename)
        binary_img.save(binary_path)

        img_filename = os.path.splitext(filename)[0] + '.jpg'
        image_path = os.path.join(input_folder, img_filename)
        gt_filename = os.path.splitext(filename)[0] + '.jpg'
        gt_path = os.path.join(gt_folder, gt_filename)

        if not os.path.exists(gt_path):
            print(f"⚠️ Ground truth mask not found for {gt_filename}, skipping visualization.")
            continue

        # Load and resize all images (grayscale)
        image = Image.open(image_path).convert('RGB').resize(image_size)
        gt = Image.open(gt_path).convert('L').resize(image_size)
        raw = raw_pred.resize(image_size)
        binary = binary_img.resize(image_size)

        images = [np.array(image), np.array(gt), np.array(raw), np.array(binary)]

        fig, axes = plt.subplots(1, 4, figsize=(12, 3))
        for ax, img, title in zip(axes, images, titles):
            # Use color map only for grayscale images
            if img.ndim == 2 or img.shape[2] == 1:
                ax.imshow(img, cmap='gray', vmin=0, vmax=255)
            else:
                ax.imshow(img)
            ax.set_title(title, fontsize=10)
            ax.axis('off')



        plt.tight_layout()
        save_path = os.path.join(comparison_folder, filename)
        plt.savefig(save_path, bbox_inches='tight', dpi=150)
        plt.close()

    print(f"✅ Saved binary predictions to {binary_folder}")
    print(f"✅ Saved visual comparisons to {comparison_folder}")


Post-processing outputs for model: u2net
✅ Saved binary predictions to /home/deepaksr/project/Saliency_datasets/RGB_only/rgb_binary_mask_pred/u2net
✅ Saved visual comparisons to /home/deepaksr/project/Saliency_datasets/RGB_only/rgb_comparison/u2net
Post-processing outputs for model: sal_v1
✅ Saved binary predictions to /home/deepaksr/project/Saliency_datasets/RGB_only/rgb_binary_mask_pred/sal_v1
✅ Saved visual comparisons to /home/deepaksr/project/Saliency_datasets/RGB_only/rgb_comparison/sal_v1
Post-processing outputs for model: sal_v2
✅ Saved binary predictions to /home/deepaksr/project/Saliency_datasets/RGB_only/rgb_binary_mask_pred/sal_v2
✅ Saved visual comparisons to /home/deepaksr/project/Saliency_datasets/RGB_only/rgb_comparison/sal_v2


In [7]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Folder paths
input_folder = '/home/deepaksr/project/Saliency_datasets/RGB_only/images'
gt_folder = '/home/deepaksr/project/Saliency_datasets/RGB_only/saliency_masks'
u2netp_folder = '/home/deepaksr/project/Saliency_datasets/RGB_only/rgb_raw_predictions/u2net'
sal_v1_folder = '/home/deepaksr/project/Saliency_datasets/RGB_only/rgb_raw_predictions/sal_v1'
sal_v2_folder = '/home/deepaksr/project/Saliency_datasets/RGB_only/rgb_raw_predictions/sal_v2'

# Output folder
comparison_grid_folder = '/home/deepaksr/project/Saliency_datasets/RGB_only/rgb_comparison_across_models'
os.makedirs(comparison_grid_folder, exist_ok=True)

# Titles for subplots
titles = ['Input Image', 'Ground Truth', 'U²-NetP', 'Saliency_v1', 'Saliency_v2']

# Image resize
image_size = (320, 320)

for filename in os.listdir(u2netp_folder):
    if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        continue

    base_name = os.path.splitext(filename)[0]
    input_path = os.path.join(input_folder, base_name + '.jpg')  # adjust extension if needed
    gt_path = os.path.join(gt_folder, base_name + '.jpg')        # adjust extension if needed
    u2netp_path = os.path.join(u2netp_folder, filename)
    sal_v1_path = os.path.join(sal_v1_folder, filename)
    sal_v2_path = os.path.join(sal_v2_folder, filename)

    if not (os.path.exists(input_path) and os.path.exists(gt_path) and 
            os.path.exists(u2netp_path) and os.path.exists(sal_v1_path) and os.path.exists(sal_v2_path)):
        print(f"⚠️ Missing files for {filename}, skipping.")
        continue

    # Load input image as RGB, others as grayscale
    input_img = np.array(Image.open(input_path).convert('RGB').resize(image_size))
    gt_img = np.array(Image.open(gt_path).convert('L').resize(image_size))
    u2netp_img = np.array(Image.open(u2netp_path).convert('L').resize(image_size))
    sal_v1_img = np.array(Image.open(sal_v1_path).convert('L').resize(image_size))
    sal_v2_img = np.array(Image.open(sal_v2_path).convert('L').resize(image_size))

    images = [input_img, gt_img, u2netp_img, sal_v1_img, sal_v2_img]

    # Create subplot
    fig, axes = plt.subplots(1, 5, figsize=(15, 3))
    for ax, img, title in zip(axes, images, titles):
        if img.ndim == 2:  # Grayscale
            ax.imshow(img, cmap='gray', vmin=0, vmax=255)
        else:  # RGB
            ax.imshow(img)
        ax.set_title(title, fontsize=9)
        ax.axis('off')

    plt.tight_layout()
    save_path = os.path.join(comparison_grid_folder, filename)
    plt.savefig(save_path, bbox_inches='tight', dpi=150)
    plt.close()

print(f"✅ Saved model comparison grids to {comparison_grid_folder}")

✅ Saved model comparison grids to /home/deepaksr/project/Saliency_datasets/RGB_only/rgb_comparison_across_models


data 5

In [35]:
# Sort the filenames to ensure consistent ordering
all_filenames = sorted([
    fname for fname in os.listdir(input_folder)
    if fname.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp'))
])

# Select one image every 150 steps
selected_filenames = all_filenames[::150]

for fname in selected_filenames:
    img_path = os.path.join(input_folder, fname)
    image = Image.open(img_path).convert(input_mode)
    input_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        if isinstance(output, (list, tuple)):
            output = output[0]

    save_path = os.path.join(model_output_folder, fname)
    save_prediction(output, save_path)


In [36]:
import os
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import numpy as np
from model import U2NET, U2NETP

# === Define Custom IR Models ===
class Saliency_IR_v1(nn.Module):
    def __init__(self):
        super(Saliency_IR_v1, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(64, 1, kernel_size=2, stride=2), nn.Sigmoid()
        )
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

class Saliency_IR_v2(nn.Module):
    def __init__(self):
        super(Saliency_IR_v2, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(8, 16, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(16, 8, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(8, 1, kernel_size=2, stride=2), nn.Sigmoid()
        )
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# === CONFIG ===
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_folder = r'/home/deepaksr/project/Saliency_datasets/IR_only/Actual scenario/images'
output_base_folder = r'/home/deepaksr/project/Saliency_datasets/IR_only/Actual scenario/raw_predictions'
image_size = (320, 320)

# === Define model loading info ===
models_info = {
    'u2net': {
        'model_class': lambda: U2NETP(1, 1),
        'weight_path': '/home/deepaksr/project/Project_files_2/U-2-Net/training_logs/u2netp_finetune_ir_d3_20250511_212158/best_model.pth',
        'input_mode': 'L'
    },
    'sal_v1': {
        'model_class': Saliency_IR_v1,
        'weight_path': '/home/deepaksr/project/Project_files_2/training_logs/IR_v1_d3_fixed_bce_20250517_132705/best_model.pth',
        'input_mode': 'L'
    },
    'sal_v2': {
        'model_class': Saliency_IR_v2,
        'weight_path': '/home/deepaksr/project/Project_files_2/training_logs/IR_v2_d3_fixed_bce_20250517_221851/best_model.pth',
        'input_mode': 'L'
    }
}

# === Transform Function ===
def get_transform(input_mode):
    return transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor()
    ])

# === Save prediction ===
def save_prediction(output_tensor, save_path):
    output = output_tensor.squeeze().cpu().detach().numpy()
    output = (output * 255).astype(np.uint8)
    Image.fromarray(output).save(save_path)

# === Inference Loop ===
for model_name, info in models_info.items():
    print(f"\n🔄 Running inference for: {model_name}")
    
    model = info['model_class']().to(device)
    model.load_state_dict(torch.load(info['weight_path'], map_location=device))
    model.eval()
    
    # Set correct image mode (L for grayscale)
    input_mode = info['input_mode']
    transform = get_transform(input_mode)

    # Prepare output directory
    model_output_folder = os.path.join(output_base_folder, model_name)
    os.makedirs(model_output_folder, exist_ok=True)

    # Sort the filenames to ensure consistent ordering
    all_filenames = sorted([
        fname for fname in os.listdir(input_folder)
        if fname.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp'))
    ])

    # Select one image every 150 steps
    selected_filenames = all_filenames[::150]

    for fname in selected_filenames:
        
    #for fname in os.listdir(input_folder):
        if fname.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp')):
            img_path = os.path.join(input_folder, fname)
            image = Image.open(img_path).convert(input_mode)
            input_tensor = transform(image).unsqueeze(0).to(device)

            with torch.no_grad():
                output = model(input_tensor)
                if isinstance(output, (list, tuple)):
                    output = output[0]

            save_path = os.path.join(model_output_folder, fname)
            save_prediction(output, save_path)

    print(f"✅ Saved predictions in {model_output_folder}")

print("\n🎉 All model predictions completed.")



🔄 Running inference for: u2net


/tmp/ipykernel_2381364/1933183954.py:91: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(info['weight_path'], map_location=device))
/home/deep

✅ Saved predictions in /home/deepaksr/project/Saliency_datasets/IR_only/Actual scenario/raw_predictions/u2net

🔄 Running inference for: sal_v1
✅ Saved predictions in /home/deepaksr/project/Saliency_datasets/IR_only/Actual scenario/raw_predictions/sal_v1

🔄 Running inference for: sal_v2
✅ Saved predictions in /home/deepaksr/project/Saliency_datasets/IR_only/Actual scenario/raw_predictions/sal_v2

🎉 All model predictions completed.


In [37]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF

# Folder paths
gt_folder = r'/home/deepaksr/project/Saliency_datasets/IR_only/Actual scenario/saliency_masks'  # Update with actual GT path
binary_base_folder = output_base_folder.replace("raw_predictions", "binary_mask_pred")
comparison_base_folder = output_base_folder.replace("raw_predictions", "comparison")

titles = ['Input Image', 'Ground Truth', 'Raw Prediction', 'Binary Prediction']

for model_name in models_info:
    print(f"Post-processing outputs for model: {model_name}")

    raw_folder = os.path.join(output_base_folder, model_name)
    binary_folder = os.path.join(binary_base_folder, model_name)
    comparison_folder = os.path.join(comparison_base_folder, model_name)

    os.makedirs(binary_folder, exist_ok=True)
    os.makedirs(comparison_folder, exist_ok=True)

    for filename in os.listdir(raw_folder):
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue

        raw_path = os.path.join(raw_folder, filename)
        raw_pred = Image.open(raw_path).convert('L')
        raw_np = np.array(raw_pred) / 255.0
        binary_np = (raw_np > 0.5).astype(np.uint8) * 255
        binary_img = Image.fromarray(binary_np)

        binary_path = os.path.join(binary_folder, filename)
        binary_img.save(binary_path)
        img_filename = os.path.splitext(filename)[0] + '.png'
        image_path = os.path.join(input_folder, img_filename)
        gt_filename = os.path.splitext(filename)[0] + '.png'
        gt_path = os.path.join(gt_folder, gt_filename)

        if not os.path.exists(gt_path):
            print(f"⚠️ Ground truth mask not found for {gt_filename}, skipping visualization.")
            continue

        # Load and resize all images (grayscale)
        image = Image.open(image_path).convert('L').resize(image_size)
        gt = Image.open(gt_path).convert('L').resize(image_size)
        raw = raw_pred.resize(image_size)
        binary = binary_img.resize(image_size)

        images = [np.array(img) for img in [image, gt, raw, binary]]

        # Create subplot grid
        fig, axes = plt.subplots(1, 4, figsize=(12, 3))
        for ax, img, title in zip(axes, images, titles):
            ax.imshow(img, cmap='gray', vmin=0, vmax=255)
            ax.set_title(title, fontsize=10)
            ax.axis('off')

        plt.tight_layout()
        save_path = os.path.join(comparison_folder, filename)
        plt.savefig(save_path, bbox_inches='tight', dpi=150)
        plt.close()

    print(f"✅ Saved binary predictions to {binary_folder}")
    print(f"✅ Saved visual comparisons to {comparison_folder}")


Post-processing outputs for model: u2net
✅ Saved binary predictions to /home/deepaksr/project/Saliency_datasets/IR_only/Actual scenario/binary_mask_pred/u2net
✅ Saved visual comparisons to /home/deepaksr/project/Saliency_datasets/IR_only/Actual scenario/comparison/u2net
Post-processing outputs for model: sal_v1
✅ Saved binary predictions to /home/deepaksr/project/Saliency_datasets/IR_only/Actual scenario/binary_mask_pred/sal_v1
✅ Saved visual comparisons to /home/deepaksr/project/Saliency_datasets/IR_only/Actual scenario/comparison/sal_v1
Post-processing outputs for model: sal_v2
✅ Saved binary predictions to /home/deepaksr/project/Saliency_datasets/IR_only/Actual scenario/binary_mask_pred/sal_v2
✅ Saved visual comparisons to /home/deepaksr/project/Saliency_datasets/IR_only/Actual scenario/comparison/sal_v2


In [38]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Assumed folder paths (update if needed)
input_folder = r'/home/deepaksr/project/Saliency_datasets/IR_only/Actual scenario/images'
gt_folder = r'/home/deepaksr/project/Saliency_datasets/IR_only/Actual scenario/saliency_masks'
u2netp_folder = r'/home/deepaksr/project/Saliency_datasets/IR_only/Actual scenario/raw_predictions/u2net'
sal_v1_folder = r'/home/deepaksr/project/Saliency_datasets/IR_only/Actual scenario/raw_predictions/sal_v1'
sal_v2_folder = r'/home/deepaksr/project/Saliency_datasets/IR_only/Actual scenario/raw_predictions/sal_v2'

# Output folder
comparison_grid_folder = r'/home/deepaksr/project/Saliency_datasets/IR_only/Actual scenario/comparison_across_models'
os.makedirs(comparison_grid_folder, exist_ok=True)

# Titles for subplots
titles = ['Input Image', 'Ground Truth', 'U²-NetP', 'Saliency_v1', 'Saliency_v2']

# Set the desired image size
image_size = (320, 320)  # Or whatever your working size is

# Loop over files in any one prediction folder (they should all match in filenames)
for filename in os.listdir(u2netp_folder):
    if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        continue

    base_name = os.path.splitext(filename)[0]

    # Paths to all versions
    input_path = os.path.join(input_folder, base_name + '.png')  # or .png if needed
    gt_path = os.path.join(gt_folder, base_name + '.png')        # or adjust extension
    u2netp_path = os.path.join(u2netp_folder, filename)
    sal_v1_path = os.path.join(sal_v1_folder, filename)
    sal_v2_path = os.path.join(sal_v2_folder, filename)

    # Check all required files exist
    if not (os.path.exists(input_path) and os.path.exists(gt_path) and 
            os.path.exists(u2netp_path) and os.path.exists(sal_v1_path) and os.path.exists(sal_v2_path)):
        print(f"⚠️ Missing files for {filename}, skipping.")
        continue

    # Load and resize all images as grayscale
    def load_gray(path):
        return np.array(Image.open(path).convert('L').resize(image_size))

    images = [load_gray(p) for p in [input_path, gt_path, u2netp_path, sal_v1_path, sal_v2_path]]

    # Create subplot
    fig, axes = plt.subplots(1, 5, figsize=(15, 3))
    for ax, img, title in zip(axes, images, titles):
        ax.imshow(img, cmap='gray', vmin=0, vmax=255)
        ax.set_title(title, fontsize=9)
        ax.axis('off')

    plt.tight_layout()
    save_path = os.path.join(comparison_grid_folder, filename)
    plt.savefig(save_path, bbox_inches='tight', dpi=150)
    plt.close()

print(f"✅ Saved model comparison grids to {comparison_grid_folder}")


✅ Saved model comparison grids to /home/deepaksr/project/Saliency_datasets/IR_only/Actual scenario/comparison_across_models
